[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-2/chatbot-external-memory.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239440-lesson-6-chatbot-w-summarizing-messages-and-external-memory)

# Chatbot with message summarization & external DB memory

## Review

We've covered how to customize graph state schema and reducer. 
 
We've also shown a number of tricks for trimming or filtering messages in graph state. 

We've used these concepts in a Chatbot with memory that produces a running summary of the conversation.

## Goals

But, what if we want our Chatbot to have **memory that persists indefinitely**?

Now, we'll introduce some more advanced **checkpointers that support external databases**. 

Here, we'll show how to use [Sqlite as a checkpointer](https://langchain-ai.github.io/langgraph/concepts/low_level/#checkpointer), but other checkpointers, such as [Postgres](https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/) for production are available!

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph-checkpoint-sqlite langchain_core langgraph langchain_openai

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

## Sqlite

A good starting point here is the [SqliteSaver checkpointer](https://langchain-ai.github.io/langgraph/concepts/low_level/#checkpointer).

Sqlite is a [small, fast, highly popular](https://x.com/karpathy/status/1819490455664685297) SQL database. 
 
If we supply `":memory:"` it creates an in-memory Sqlite database.


In [3]:
import sqlite3
# In memory
conn = sqlite3.connect(":memory:", check_same_thread = False)

But, if we supply a db path, then it will create a database for us!

In [4]:
# pull file if it doesn't exist and connect to local db
!mkdir -p state_db && [ ! -f state_db/example.db ] && wget -P state_db https://github.com/langchain-ai/langchain-academy/raw/main/module-2/state_db/example.db

db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)

> **Note on `check_same_thread=False`**  
> By default, Python’s `sqlite3` enforces that a database connection can only be used in the same thread where it was created.  

> Setting `check_same_thread=False` allows the same connection to be shared across threads, which is required in LangGraph/LangChain setups where multiple nodes or async tasks may access the database.
  
> This does not make SQLite fully thread-safe — concurrency is handled at a higher level by the framework.

In [5]:
# Here is our checkpointer 
from langgraph.checkpoint.sqlite import SqliteSaver
memory = SqliteSaver(conn)

Let's re-define our chatbot.

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
    
    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above:"

    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State):
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

Now, we just re-compile with our sqlite checkpointer.

In [9]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
graph = workflow.compile(checkpointer=memory) # memory = SqliteSaver(conn); conn = sqlite3.connect(db_path, check_same_thread=False)
#display(Image(graph.get_graph().draw_mermaid_png()))

Now, we can invoke the graph several times. 

In [10]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Start conversation
input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hello again, Lance! It's great to see you back. Is there something specific you'd like to chat about today? Maybe more about the 49ers or something else on your mind?
================================== Ai Message ==================================

Your name is Lance! How can I assist you today?
================================== Ai Message ==================================

That's awesome, Lance! The 49ers have a rich history and a passionate fan base. Is there a particular aspect of the team you enjoy discussing, like their current roster, memorable games, or maybe their prospects for the upcoming season?


Let's confirm that our state is saved locally.

In [11]:
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Lance", additional_kwargs={}, response_metadata={}, id='875b0e70-8a31-4b34-bab0-29a21f5d5ade'), AIMessage(content="Hello again, Lance! It's great to see you back. Is there something specific you'd like to chat about today? Maybe more about the 49ers or something else on your mind?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 337, 'total_tokens': 374, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CJo4RTSskHsFQaVls5OyNwX2WX2vT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--abf6c7bb-40ef-45ef-b3cf-7b7fd635b06a-0', usage_metadata={'input_tokens': 337, 'output_tokens': 37,

### Persisting state

##### Using database like Sqlite means state is persisted! 

For example, we can **re-start the notebook kernel and see that we can still load from Sqlite DB on disk.**


In [3]:
# 1) Reconnect to the SAME SQLite DB (on disk, not in memory)
import sqlite3
db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
print("Connected to:", db_path)

# 2) Rebuild the checkpointer + graph (same as before restart)
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import HumanMessage, AIMessage

checkpointer = SqliteSaver(conn)

class State(MessagesState):
    pass

def echo_node(state: State):
    return {"messages": [AIMessage(content=f"Echo: {state['messages'][-1].content}")]}

builder = StateGraph(State)
builder.add_node("echo", echo_node)
builder.add_edge(START, "echo")
builder.add_edge("echo", END)

graph = builder.compile(checkpointer=checkpointer)

# 3) Load persisted state (use same thread_id you wrote before restart)
config = {"configurable": {"thread_id": "1"}}
snap = graph.get_state(config)
print("\nSnapshot loaded from SQLite:")
print(snap)

if snap.values.get("messages"):
    print("\nRecovered messages:")
    for m in snap.values["messages"]:
        m.pretty_print()
else:
    print("No messages found for this thread_id. Did you persist any before restart?")

# 4) Inspect the raw checkpoints table for proof
import json
cur = conn.cursor()
cur.execute("""
SELECT thread_id, MAX(checkpoint_ns) as latest_ns, LENGTH(checkpoint) as size_bytes
FROM checkpoints
GROUP BY thread_id
ORDER BY latest_ns DESC;
""")
print("\nLatest checkpoint per thread:")
for row in cur.fetchall():
    print(row)


Connected to: state_db/example.db

Snapshot loaded from SQLite:
StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Lance", additional_kwargs={}, response_metadata={}, id='875b0e70-8a31-4b34-bab0-29a21f5d5ade'), AIMessage(content="Hello again, Lance! It's great to see you back. Is there something specific you'd like to chat about today? Maybe more about the 49ers or something else on your mind?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 337, 'total_tokens': 374, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CJo4RTSskHsFQaVls5OyNwX2WX2vT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--abf6c7bb-40ef-45ef-b3cf-7b7fd635b0

In [4]:
import sqlite3

db_path = "state_db/example.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
cur = conn.cursor()

# 1) Show all column names for the checkpoints table
cur.execute("PRAGMA table_info(checkpoints);")
print("Columns in checkpoints table:")
for row in cur.fetchall():
    print(row)

# 2) Select all rows (be careful if the table is very large)
print("\nAll rows in checkpoints table:")
cur.execute("SELECT * FROM checkpoints;")
rows = cur.fetchall()
for r in rows:
    print(r)

Columns in checkpoints table:
(0, 'thread_id', 'TEXT', 1, None, 1)
(1, 'checkpoint_ns', 'TEXT', 1, "''", 2)
(2, 'checkpoint_id', 'TEXT', 1, None, 3)
(3, 'parent_checkpoint_id', 'TEXT', 0, None, 0)
(4, 'type', 'TEXT', 0, None, 0)
(5, 'checkpoint', 'BLOB', 0, None, 0)
(6, 'metadata', 'BLOB', 0, None, 0)

All rows in checkpoints table:
('1', '', '1ef64b22-3155-65d8-bfff-cb35a53ab872', None, 'json', b'{"v": 1, "ts": "2024-08-27T20:22:52.733166+00:00", "id": "1ef64b22-3155-65d8-bfff-cb35a53ab872", "channel_values": {"messages": [], "__start__": {"messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "hi! I\'m Lance", "type": "human"}}]}}, "channel_versions": {"__start__": "00000000000000000000000000000001.761a8c54329ca020d4813f3f8164401b"}, "versions_seen": {"__input__": {}}, "pending_sends": [], "current_tasks": {}}', b'{"source": "input", "writes": {"messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "sc

## LangGraph Studio

**⚠️ DISCLAIMER**

Since the filming of these videos, we've updated Studio so that it can be run locally and opened in your browser. This is now the preferred way to run Studio (rather than using the Desktop App as shown in the video). See documentation [here](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server) on the local development server and [here](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server). To start the local development server, run the following command in your terminal in the `/studio` directory in this module:

```
langgraph dev
```

You should see the following output:
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

Open your browser and navigate to the Studio UI: `https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`.

#### Langgraph Studio has built-in persistence using Postgres !!!